In [41]:
import pandas as pd

In [42]:
'''
This Notebook focuses on using pandas to automate portions of the reporting process. Specifically, the transformation
of accounting Journal Entries (JE) into meaningful Financial Statements.

The data is being sourced from academic repositories (MPA level worksheets).

To Note: Natural Language Processing Algorithms are NOT recommended
when it comes to analyzing free-form text transaction logs. If these serve as the basis for creating JE, the accuracy
needs to be perfect to avoid financial liabilities.
'''

'\nThis Notebook focuses on using pandas to automate portions of the reporting process. Specifically, the transformatin\nof accounting Journal Entries (JE) into meaningful Financial Statements.\n\nThe data is being sourced from academic repositories (MPA level worksheets).\n\nTo Note: Natural Language Processing Algorithms are NOT recommended\nwhen it comes to analyzing free-form text transaction logs. If these serve as the basis for creating JE, the accuracy\nneeds to be perfect to avoid financial liabilities.\n'

In [71]:
# Load the data into a local dataframe
JE = pd.read_excel('./JournalEntriesData.xlsx')
display (JE.head())

,Type,Description,Journal Entry,Debit/Credit
0,Comprehensive Examples,Tawhoe Oil Company was interested in a large t...,G&G Expense\nCash,50000\n50000
1,Comprehensive Examples,"Tawhoe paid a geological firm $500,000 to cond...",G&G Expense\nCash,500000\n500000
2,Comprehensive Examples,"Based on the results of that study, Tawhoe acq...",G&G Expense\nCash,1500000\n1500000
3,Comprehensive Examples,"During the first year, Tawhoe had to pay $15,0...","Ad valorem tax expense, Legal expense - explor...","15000, 16000\n15000, 16000"
4,Comprehensive Examples,Land Acquisition - Unproved,Land Acquisition - Unproved\nCash,62000\n62000


In [44]:
'''
In order to translate the data into meaningful insights for use in the balance sheet, we must first parse the data in the
journal entry. Here, it's been divided between line items with commas, and between debit/credit by 'new lines'.
Adapt the parsing logic as needed. The key is consistency in the source data.
'''

"\nIn order to translate the data into meaningful insights for use in the balance sheet, we must first parse the data in the\njournal entry. Here, it's been divided between line items with commas, and between debit/credit by 'new lines'.\nAdapt the parsing logic as needed. The key is consistency in the source data.\n"

In [93]:
# First, create a new dataframe containing only the needed items. Each line should be a debit or credit, one item.
df = pd.DataFrame(columns = ['Debit/Credit','Line Item','Amount'])
print (df)

Empty DataFrame
Columns: [Debit/Credit, Line Item, Amount]
Index: []


In [94]:
# Do some exploration as to how the dataframe is structured
# The 'item' below is a tuple, with the first item being the index and the second being the actual data (dictionary)
for item in JE.iterrows():
    index = item[0]
    data = item[1]
    print ('Index: ', index )
    print (data)
    break

Index:  0
Type                                        Comprehensive Examples
Description      Tawhoe Oil Company was interested in a large t...
Journal Entry                                    G&G Expense\nCash
Debit/Credit                                          50000\n50000
Name: 0, dtype: object


In [95]:
# Iterate through the entire dataframe now.
for item in JE.iterrows():
    index = item[0]
    data = item[1]
    journalEntry = data['Journal Entry']
    debitCredit = data['Debit/Credit']
    # Try to split the journalEntry and debitCredit by 'new line character' to break between debit and credits
    debitDescription = journalEntry.split('\n')[0]
    creditDescription = journalEntry.split('\n')[1]
    debitAmounts = debitCredit.split('\n')[0]
    creditAmounts = debitCredit.split('\n')[1]
    # Now that we've split debit and credits, we need to iterate through the line items, which are separated by commas
    # First comes the debits
    for i in range (0, len(debitDescription.split(','))):
        debitAmountItem = debitAmounts.split(',')
        df = df.append({'Debit/Credit': 'Debit', 'Line Item': debitDescription.split(',')[i], 'Amount': debitAmountItem[i] }, ignore_index=True)
    # Do the same for credits
    for i in range (0, len(creditDescription.split(','))):
        creditAmountItem = creditAmounts.split(',')
        df = df.append({'Debit/Credit': 'Credit', 'Line Item': creditDescription.split(',')[i], 'Amount': creditAmountItem[i] }, ignore_index=True)
print ('Number of line item: ',len(df))
display (df[:10])
    

Number of line item:  82


,Debit/Credit,Line Item,Amount
0,Debit,G&G Expense,50000
1,Credit,Cash,50000
2,Debit,G&G Expense,500000
3,Credit,Cash,500000
4,Debit,G&G Expense,1500000
5,Credit,Cash,1500000
6,Debit,Ad valorem tax expense,15000
7,Debit,Legal expense - exploration,16000
8,Credit,Cash,15000
9,Credit,Cash,16000


In [96]:
# Now that we have a logical list of line item transactions, let's see all the unique categories. This will
# help consolidate them for the balance sheet.
# Let's also clean up the data, since it looks like there are some leading and trailing whitespaces.
def whiteSpaceParse(x):
    if x[0] == ' ':
        x = x[1:]
    if x[-1] == ' ':
        x = x[:-1]
    return (x.title())
df['Line Item'] = df['Line Item'].apply(whiteSpaceParse)
for item in (df['Line Item'].unique()):
    print (item)

G&G Expense
Cash
Ad Valorem Tax Expense
Legal Expense - Exploration
Land Acquisition - Unproved
Delay Rental Expense
Test-Well Contribution Expense
Unproved Property
Wells-In-Progress(Exploratory)
Dry-Hole Expense
Wells And Equipment
Proved Property
Production Expense
Wells-In-Progress - Idc
Wells-In-Progress - Lease And Well Equipment (L&We)
Wells And Equipment - Idc
Wells And Equipment - L&We
Dry-Hole Expense - Idc
Dry-Hole Expense - L&We


In [106]:
# It's finally time to consolidate all the changes to get a single view of what items on the balance sheet are changing
# and by how much
Change = {}
for item in df['Line Item'].unique():
    filteredDF = df[df['Line Item'] == item]
    totalChange = 0
    for row in filteredDF.iterrows():
        if row[1]['Debit/Credit'] == 'Debit':
            totalChange += int(row[1]['Amount'])
        else:
            totalChange -= int(row[1]['Amount'])
    Change[item] = totalChange
for item in Change:
    print (item+': $'+str(Change[item]))
        

G&G Expense: $3040000
Cash: $-9686200
Ad Valorem Tax Expense: $15000
Legal Expense - Exploration: $16000
Land Acquisition - Unproved: $62000
Delay Rental Expense: $16200
Test-Well Contribution Expense: $200000
Unproved Property: $0
Wells-In-Progress(Exploratory): $0
Dry-Hole Expense: $1300000
Wells And Equipment: $2375000
Proved Property: $82000
Production Expense: $75000
Wells-In-Progress - Idc: $-1705000
Wells-In-Progress - Lease And Well Equipment (L&We): $-270000
Wells And Equipment - Idc: $1890000
Wells And Equipment - L&We: $595000
Dry-Hole Expense - Idc: $1725000
Dry-Hole Expense - L&We: $270000


In [98]:
'''
The reason why so many changes result in $0 is because a lot of the exploration gets cancelled out when 
transitioning expenses into assets and vice-versa (ie: when we debit wells-in-progress and credit cash,
the change is reverted because we later on credit wells-in-progress and debit wells and equipment). This is likely
for reporting or tax purposes.
'''

In [113]:
# To finish, let's imagine we have a dictionary of mapped values that connects the line item type to either an asset or Liability
# For the time being, just create one. 
lineItemMapping = {'G&G Expense': 'Liability',
                  'Cash': "Asset",
                  "Ad Valorem Tax Expense": 'Liability',
                  "Legal Expense - Exploration": "Liability",
                  "Land Acquisition - Unproved": "Asset",
                  "Delay Rental Expense": "Liability",
                  'Test-Well Contribution Expense': "Liability",
                  "Unproved Property": "Asset",
                  "Wells-In-Progress(Exploratory)": "Asset",
                  "Dry-Hole Expense":"Liability",
                  "Wells And Equipment": 'Asset',
                  'Proved Property': 'Asset',
                  'Production Expense': 'Liability',
                  'Wells-In-Progress - Idc': 'Asset',
                  'Wells-In-Progress - Lease And Well Equipment (L&We)': "Asset",
                  'Wells And Equipment - Idc': 'Asset',
                  'Wells And Equipment - L&We': 'Asset',
                  'Dry-Hole Expense - Idc': 'Liability',
                  'Dry-Hole Expense - L&We': "Liability"}

In [116]:
totalAsset = 0
totalLiability = 0
for item in Change:
    if lineItemMapping[item] == "Asset":
        totalAsset += Change[item]
    else:
        totalLiability -= Change[item]
print ('Total Change in Assets:', totalAsset)
print ('Total Change in Liabilities:', totalLiability)
if (totalAsset == totalLiability):
    print ('Total changes in assets and liabilities equal, meaning there was no miscalculation!')

Total Change in Assets: -6657200
Total Change in Liabilities: -6657200
Total changes in assets and liabilities equal, meaning there was no miscalculation!
